# Goal of this notebook

- use certain metrics (LM perplexity, G2P error) to identify wordtypes that are more likely to be mispronounced by our grapheme-input TTS system

# automatic reloading magic

In [128]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports 

In [170]:
import json
from datasets import load_dataset
import torch
import jiwer
from tqdm import tqdm

# check if have correct type of node 

In [130]:
import socket

# print hostname to make sure we are on correct node
disallowed_nodes = ['escience6']
hostname = socket.gethostname()
print(hostname)
node = hostname.split('.')[0]
if node in disallowed_nodes:
    raise ValueError(f"Running on disallowed node {node}!")

barre.inf.ed.ac.uk


In [131]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()
assert torch.cuda.is_available()

# Load OOV list

(wordtypes not seen in the half of LJSpeech used to train TTS and ASR)

In [132]:
# get oov wordtypes list (words that are not seen in tts training)
oov_wordlist_path = '/home/s1785140/data/ljspeech_fastpitch/oov_list.json'
with open(oov_wordlist_path, 'r') as f:
    oovs_and_freqs = json.load(f)
    
oovs = set(w.strip() for w in oovs_and_freqs.keys())
print(f'original before cleaning/sampling {len(oovs)=}')

original before cleaning/sampling len(oovs)=8343


# Load G2P pronunciation lexicon

In [133]:
# load lexicon that G2P model was trained on
dataset_dict = load_dataset("flexthink/librig2p-nostress")

Found cached dataset librig2p-nostress (/home/s1785140/.cache/huggingface/datasets/flexthink___librig2p-nostress/default/0.0.0/95c204c6be42796a753ef410b5dfce2bfa21d61b51f0c3ffe85cf6e3a4dee65f)


  0%|          | 0/6 [00:00<?, ?it/s]

In [134]:
type(dataset_dict['lexicon_train'])

datasets.arrow_dataset.Dataset

In [135]:
# combine data splits in the lexicon 
# (as we are not training G2P, don't need train valid test splits)
from datasets import concatenate_datasets
datasets = [dataset_dict['lexicon_train'], dataset_dict['lexicon_valid'], dataset_dict['lexicon_test']]
dataset = concatenate_datasets(datasets)

In [136]:
# create dict of wordtype to pronunciation
lexicon = {}
for char, phn in zip(dataset['char'], dataset['phn']):
    lexicon[char.lower().strip()] = phn

# only consider the OOV words that are in the pronunciation lexicon


In [137]:
len(oovs)

8343

In [138]:
oovs_in_lexicon = oovs.intersection(lexicon.keys())
len(oovs_in_lexicon)

7966

In [139]:
oovs_not_in_lexicon = oovs - oovs_in_lexicon
len(oovs_not_in_lexicon)

377

# Compare LibriG2P coverage against CMUdict

In [140]:
!pip install cmudict

In [141]:
import cmudict as cmudict_module

In [142]:
cmudict = cmudict_module.dict()

## normalise cmudict entries

- key: all lowercase
- values: phone strings should not hav any stress markers

In [143]:
for wordtype in cmudict.keys():
    if wordtype.isupper():
        raise ValueError
else:
    print("Keys do not contain any capital letters")

Keys do not contain any capital letters


In [144]:
prons = cmudict['content']

In [145]:
prons

[['K', 'AA1', 'N', 'T', 'EH0', 'N', 'T'],
 ['K', 'AH0', 'N', 'T', 'EH1', 'N', 'T']]

In [146]:
def strip_stress(pron):
    stripped_pron = []
    for phn in pron:
        stripped_pron.append(phn.strip('0123456789'))
    return stripped_pron

strip_stress(['K', 'AA1', 'N', 'T', 'EH0', 'N', 'T'])

['K', 'AA', 'N', 'T', 'EH', 'N', 'T']

In [147]:
new_cmudict = {}
for wordtype, prons in cmudict.items():
    new_prons = []
    for pron in prons:
        new_prons.append(strip_stress(pron))
    new_cmudict[wordtype] = new_prons

In [148]:
cmudict = new_cmudict

In [149]:
cmudict['content']

[['K', 'AA', 'N', 'T', 'EH', 'N', 'T'], ['K', 'AH', 'N', 'T', 'EH', 'N', 'T']]

## find words in OOV list that are in CMUDict

In [150]:
len(oovs)

8343

In [151]:
oovs_in_cmudict = oovs.intersection(cmudict.keys())
len(oovs_in_cmudict)

7233

In [152]:
oovs_not_in_cmudict = oovs - oovs_in_cmudict
len(oovs_not_in_cmudict)

1110

## find OOVs not in LibriG2P or cmudict

In [153]:
oovs_not_in_cmudict_and_librig2p = oovs_not_in_cmudict.intersection(oovs_not_in_lexicon)
len(oovs_not_in_cmudict_and_librig2p)

274

In [154]:
oovs_not_in_cmudict_and_librig2p

{'accordez',
 'adaptively',
 'affectionless',
 'afterwork',
 'agardh',
 'agencys',
 'agonal',
 'akermans',
 'akkad',
 'aldermens',
 'alinement',
 'aloman',
 'amuhia',
 'amylaceous',
 'anabolism',
 'arachtu',
 'argool',
 'arke',
 'armys',
 'arthor',
 'askern',
 'baddow',
 'bagne',
 'bamell',
 'barbariously',
 'bashour',
 'batess',
 'bbl',
 'belian',
 'billfolds',
 'bodoni',
 'bouhes',
 'bradawls',
 'bringuiers',
 'bubbletop',
 'buranelli',
 'busdriver',
 'buxtons',
 'cabmans',
 'caducibranchs',
 'cannings',
 'cardiotachyscope',
 'catwalks',
 'caunts',
 'centurys',
 'cerebrospinal',
 'chaldasan',
 'charae',
 'chiselers',
 'chromatin',
 'chromatophores',
 'chummage',
 'cissian',
 'citys',
 'cleancutness',
 'clipperton',
 'collaborates',
 'colsman',
 'compters',
 'conceptualisation',
 'condigne',
 'connallys',
 'conveners',
 'countrys',
 'courtmartialed',
 'crosshair',
 'cutdowns',
 'cyruss',
 'daulby',
 'daulbys',
 'delarues',
 'delustered',
 'deoxidation',
 'detre',
 'diemens',
 'dixblan

In [155]:
!pip install num2words

In [156]:
import num2words
from num2words import num2words

In [157]:
num2words(25)

'twenty-five'

In [158]:
nums = [num2words(i).strip('-') for i in range(100)]

In [159]:
def does_not_contain_num(s):
    for num in nums:
        if num in s:
            return False
    return True

In [160]:
## filter this list 



# word len
print(f"111 {len(oovs_not_in_cmudict_and_librig2p)=}")
oovs_not_in_cmudict_and_librig2p = set(w for w in oovs_not_in_cmudict_and_librig2p if len(w) >= 4)
print(f"222 {len(oovs_not_in_cmudict_and_librig2p)=}")

# not a number
oovs_not_in_cmudict_and_librig2p = set(w for w in oovs_not_in_cmudict_and_librig2p if does_not_contain_num(w))
print(f"333 {len(oovs_not_in_cmudict_and_librig2p)=}")

111 len(oovs_not_in_cmudict_and_librig2p)=274
222 len(oovs_not_in_cmudict_and_librig2p)=268
333 len(oovs_not_in_cmudict_and_librig2p)=246


# generate pronunciation for each word in the data used to train ASR/TTS

In [161]:
train_data = '/home/s1785140/data/ljspeech_fastpitch/train_meta_half.txt'
with open(train_data, 'r') as f:
    train_data = f.readlines()

In [162]:
train_utts = [l.split('|')[-1] for l in train_data]

In [163]:
# use text cleaner to clean each utt

from fastpitch.common.text.cleaners import lowercase_no_punc as text_cleaner

train_utts = [text_cleaner(utt) for utt in train_utts]


In [164]:
train_wordtypes = set()
for utt in train_utts:
    for token in utt.split(' '):
        train_wordtypes.add(token)
len(train_wordtypes)

5612

In [165]:
train_wordtypes

{'friend',
 'wear',
 'voebel',
 'exit',
 'surveys',
 'procured',
 'portions',
 'extreme',
 'be',
 'crowded',
 'thieves',
 'elbow',
 'signature',
 'acquitted',
 'medicine',
 'limited',
 'my',
 'sister',
 'accomplices',
 'owners',
 'retired',
 'led',
 'governors',
 'crying',
 'traders',
 'britain',
 'fpcc',
 'homologies',
 'evaded',
 'concourse',
 'wall',
 'june',
 'four',
 'billions',
 'doors',
 'latona',
 'daughters',
 'poisoning',
 'barrel',
 'exact',
 'donnell',
 'religious',
 'honestly',
 'enforcement',
 'monies',
 'bureau',
 'indignation',
 'rendered',
 'kleins',
 'build',
 'doubted',
 'wonder',
 'adduced',
 'fluid',
 'profound',
 'operation',
 'gordon',
 'appointed',
 'alan',
 'lancaster',
 'temple',
 'worn',
 'lads',
 'wagner',
 'belmont',
 'attempting',
 'about',
 'enforced',
 'allegations',
 'prisoners',
 'house',
 'warning',
 'renewed',
 'york',
 'fingers',
 'books',
 'threw',
 'tendencies',
 'defensive',
 'inscription',
 'ago',
 'badly',
 'cuba',
 'appear',
 'arsenic',
 'bran

In [166]:
print("there are ", len(train_wordtypes.intersection(set(cmudict.keys())))," training wordtypes in cmudict")

there are  5431  training wordtypes in cmudict


In [174]:
# use cmudict to get pronunciation for train wordtypes

train_wordtype2pron = {}
for w in train_wordtypes.intersection(set(cmudict.keys())):
    train_wordtype2pron[w] = cmudict[w][0]

In [175]:
# use G2P to predict pronunciations for wordtypes in training set but not in cmudict

from speechbrain.pretrained import GraphemeToPhoneme
g2p = GraphemeToPhoneme.from_hparams("speechbrain/soundchoice-g2p")

In [176]:
for w in tqdm(train_wordtypes - set(cmudict.keys())):
    train_wordtype2pron[w] = g2p(w)

  0%|                                                             | 0/181 [00:00<?, ?it/s]Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████| 181/181

## save pron dict to disk

in two formats
- generic
- for training G2P model (https://pypi.org/project/phonetisaurus/)

In [178]:
# generic
with open('/home/s1785140/data/ljspeech_fastpitch/train_meta_half_pron_dict.json', 'w') as f:
    json.dump(train_wordtype2pron, f)

In [181]:
# for G2P training
#/path/to/lexicon.dict
#lexicon format:
#word1 phoneme1 phoneme2 ...
#word2 phoneme1 phoneme2 phoneme3 ...
g2p_training_lexicon_lines = []
for w, pron in train_wordtype2pron.items():
    l = f"{w} {' '.join(pron)}"
    g2p_training_lexicon_lines.append(l)

In [184]:
g2p_training_lexicon_lines[0]

'friend F R EH N D'

In [185]:
with open('/home/s1785140/data/ljspeech_fastpitch/train_meta_half_prons_for_training_g2p.dict', 'w') as f:
    f.write('\n'.join(g2p_training_lexicon_lines))

# Train G2P model using only wordtypes contained in data used to train TTS/ASR

In [187]:
!pip install phonetisaurus

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 4.1 MB/s eta 0:00:0000:0100:01


In [188]:
import subprocess
subprocess.run(["phonetisaurus",
                "train",
                "--model", 
                "/home/s1785140/data/ljspeech_fastpitch/train_meta_half_phonetisaurusG2P_model.fst",
                "/home/s1785140/data/ljspeech_fastpitch/train_meta_half_prons_for_training_g2p.dict"])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:phonetisaurus-train:2023-03-15 16:08:51:  Checking command configuration...
DEBUG:phonetisaurus-train:2023-03-15 16:08:51:  Directory does not exist.  Trying to create.
INFO:phonetisaurus-train:2023-03-15 16:08:51:  Checking lexicon for reserved characters: '}', '|', '_'...
DEBUG:phonetisaurus-train:2023-03-15 16:08:51:  arpa_path:  train/model.o8.arpa
DEBUG:phonetisaurus-train:2023-03-15 16:08:51:  corpus_path:  train/model.corpus
DEBUG:phonetisaurus-train:2023-03-15 16:08:51:  dir_prefix:  train
DEBUG:phonetisaurus-train:2023-03-15 16:08:51:  grow:  False
DEBUG:phonetisaurus-train:2023-03-15 16:08:51:  lexicon_file:  /tmp/tmp5lwpye_m.txt
DEBUG:phonetisaurus-train:2023-03-15 16:08:51:  logger:  <Logger phonetisaurus-train (DEBUG)>
DEBUG:phonetisaurus-train:2023-03-15 16:08:51:  makeJointNgramCommand:  <bound method G2PModelTrainer._mitlm of <__main__.G2PModelTrainer object at 0x7f6608dc4580>>
DEBUG:phonetisaurus-train:2023-03-15 16:08:51:  model_path:  train/model.fst
DEBUG:phone

CompletedProcess(args=['phonetisaurus', 'train', '--model', '/home/s1785140/data/ljspeech_fastpitch/train_meta_half_phonetisaurusG2P_model.fst', '/home/s1785140/data/ljspeech_fastpitch/train_meta_half_prons_for_training_g2p.dict'], returncode=1)